In [0]:
!pip install pandas
!pip install tweepy
!pip install vaderSentiment

In [1]:
import tweepy
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [20]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
from textblob import TextBlob
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [43]:
#My Twitter API Authentication Variables
consumer_key = 'xx'
consumer_secret = 'xx'
access_token = 'xx'
access_token_secret = 'xx'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

tweets = api.search('India', count=200)


data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

display(data.head(10))

,Tweets
0,RT @ZeeNewsHindi: VIDEO: जब विंग कमांडर अभिनंद...
1,RT @Republic_Bharat: UNSC से आज भारत को मिल सक...
2,"RT @PMOIndia: Dr. Saud Mohammed Al-Sati, Saudi..."
3,RT @Shakra_Kulish: Mohan Das Karam Chand Gandh...
4,@QararaRasha Life is complex and so is interna...
5,@bilalahmed481 That is Pakistani occupied kahm...
6,Welcome to India wc abhinandan sir
7,@swiftsIytherin yes listen remember when Pakis...
8,RT @CarefreeMunda: Jab se hamare mulk walo ne ...
9,RT @Babu_Bhaiyaa: India entered Pak for reveng...


In [29]:
new_tweets = api.user_timeline(screen_name="MKBHD", count=200)

In [30]:
for tweet in new_tweets[:5]:
    print(tweet.text)

My Galaxy S10 just got the much anticipated update: I can now remap the Bixby button to open any app. Except Google… https://t.co/YZT51q3KC7
@elonmusk Sick matte black profile picture
RT @ElectrekCo: Tesla finally launches base Model 3 for $35,000 with shorter range and new interior https://t.co/jLadgmRoeA by @fredericlam…
@UnboxTherapy @Mrwhosetheboss @kwon_ji_hae @UrAvgConsumer @Dave2D @DetroitBORG @superscientific @ijustine… https://t.co/ubFvsZNObk
WELL THEN. Tesla just paused all orders and now shows this on their site instead. I smell a Model S/X interior refr… https://t.co/UIaVXBEczR


In [31]:
new_data = pd.DataFrame(data=[tweet.text for tweet in new_tweets], columns=['Tweets'])

In [32]:
new_data['len']  = np.array([len(tweet.text) for tweet in new_tweets])
new_data['ID']   = np.array([tweet.id for tweet in new_tweets])
new_data['Date'] = np.array([tweet.created_at for tweet in new_tweets])
new_data['Source'] = np.array([tweet.source for tweet in new_tweets])
new_data['Likes']  = np.array([tweet.favorite_count for tweet in new_tweets])
new_data['RTs']    = np.array([tweet.retweet_count for tweet in new_tweets])

In [33]:
display(new_data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs
0,My Galaxy S10 just got the much anticipated up...,140,1101271053295411206,2019-03-01 00:00:54,Twitter for Android,9190,423
1,@elonmusk Sick matte black profile picture,42,1101243060984467457,2019-02-28 22:09:40,Flamingo for Android,6880,108
2,RT @ElectrekCo: Tesla finally launches base Mo...,140,1101240993498783744,2019-02-28 22:01:27,Flamingo for Android,0,698
3,@UnboxTherapy @Mrwhosetheboss @kwon_ji_hae @Ur...,130,1101237827369660416,2019-02-28 21:48:52,Twitter for Android,3080,66
4,WELL THEN. Tesla just paused all orders and no...,140,1101224617820241920,2019-02-28 20:56:23,Tweetbot for Mac,8248,481
5,@PatFlynn 🙏 Was great meeting you guys!,39,1101206714471378945,2019-02-28 19:45:14,Tweetbot for Mac,31,0
6,@CaseyNeistat @RickShielsPGA @RocketHosel @_Co...,95,1101198485100085248,2019-02-28 19:12:32,Twitter for Android,1274,9
7,@RickShielsPGA @CaseyNeistat @RocketHosel @_Co...,101,1101195926000013317,2019-02-28 19:02:22,Tweetbot for Mac,335,0
8,Ultrawide NYC from Galaxy S10+\n\nStill Samsun...,111,1101157879594303488,2019-02-28 16:31:11,Twitter for Android,18668,1215
9,@Jaime_Rivera @WhatADayDomi @Pocketnow No worr...,56,1101110907575390208,2019-02-28 13:24:32,Twitter for Android,221,2


In [36]:
def clean_tweet1(tweet):
    x = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    return x

In [37]:
for tweet in new_data['Tweets']:
    a = clean_tweet1(tweet)
    print(a)

My Galaxy S10 just got the much anticipated update I can now remap the Bixby button to open any app Except Google
Sick matte black profile picture
RT Tesla finally launches base Model 3 for 35 000 with shorter range and new interior by
ji hae
WELL THEN Tesla just paused all orders and now shows this on their site instead I smell a Model S X interior refr
Was great meeting you guys
Coxy2017 MY DEPTH PERCEPTION WAS OFF
Coxy2017 I know my way around a course hah
Ultrawide NYC from Galaxy S10 Still Samsung Still overexposed everything Still fun
Rivera No worries at all
What a good boi
Not even subtle lol
touch
Rivera Heh that frame of my video has been passed around so much it s hard to keep track lol
Shoutout to the truck drivers for making it possible lol
a g86 I m literally perfectly in between the white lines that this poorly painted parking lot officially uses lol
I have so many witnesses to this it s silly
From my IG story Pretty sure these trucks were trolling me Thanks anyway
Not u

In [38]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def sentiment_analyser(tweet):
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [39]:
new_data['SA'] = np.array([ sentiment_analyser(tweet) for tweet in new_data['Tweets'] ])

In [41]:
pos_tweets = [ tweet for index, tweet in enumerate(new_data['Tweets']) if new_data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(new_data['Tweets']) if new_data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(new_data['Tweets']) if new_data['SA'][index] < 0]

In [42]:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(new_data['Tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(new_data['Tweets'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(new_data['Tweets'])))

Percentage of positive tweets: 36.5%
Percentage of neutral tweets: 51.5%
Percentage de negative tweets: 12.0%


In [40]:
new_data.head(10)

,Tweets,len,ID,Date,Source,Likes,RTs,SA
0,My Galaxy S10 just got the much anticipated up...,140,1101271053295411206,2019-03-01 00:00:54,Twitter for Android,9190,423,1
1,@elonmusk Sick matte black profile picture,42,1101243060984467457,2019-02-28 22:09:40,Flamingo for Android,6880,108,-1
2,RT @ElectrekCo: Tesla finally launches base Mo...,140,1101240993498783744,2019-02-28 22:01:27,Flamingo for Android,0,698,-1
3,@UnboxTherapy @Mrwhosetheboss @kwon_ji_hae @Ur...,130,1101237827369660416,2019-02-28 21:48:52,Twitter for Android,3080,66,0
4,WELL THEN. Tesla just paused all orders and no...,140,1101224617820241920,2019-02-28 20:56:23,Tweetbot for Mac,8248,481,0
5,@PatFlynn 🙏 Was great meeting you guys!,39,1101206714471378945,2019-02-28 19:45:14,Tweetbot for Mac,31,0,1
6,@CaseyNeistat @RickShielsPGA @RocketHosel @_Co...,95,1101198485100085248,2019-02-28 19:12:32,Twitter for Android,1274,9,0
7,@RickShielsPGA @CaseyNeistat @RocketHosel @_Co...,101,1101195926000013317,2019-02-28 19:02:22,Tweetbot for Mac,335,0,0
8,Ultrawide NYC from Galaxy S10+\n\nStill Samsun...,111,1101157879594303488,2019-02-28 16:31:11,Twitter for Android,18668,1215,1
9,@Jaime_Rivera @WhatADayDomi @Pocketnow No worr...,56,1101110907575390208,2019-02-28 13:24:32,Twitter for Android,221,2,0


In [4]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\nagra\AppData\Roaming\nltk_data...


True

In [5]:
sid = SentimentIntensityAnalyzer()


listy = []
sentiment = []

for index, row in data.iterrows():
  ss = sid.polarity_scores(row["Tweets"])
  #print(ss)
  
  for i in ss:
    if i=='neg':
      if ss[i] > 0.5:
        polarity = 'negative'
    elif i=='neu':
      if ss[i] > 0.5:
        polarity = 'neutral'
    elif i=='pos':
      if ss[i] > 0.5:
        polarity = 'positive'
    else:
      if ss[i] > 100:
        polarity = 'Compound'
    
  
  listy.append(ss)
  sentiment.append(polarity)
  


In [35]:
#polarity = null
for i in ss:
  if i=='neg':
    if ss[i] > 0.5:
      polarity = 'negative'
  elif i=='neu':
    if ss[i] > 0.5:
      polarity = 'neutral'
  elif i=='pos':
    if ss[i] > 0.5:
      polarity = 'positive'
  else:
    if ss[i] > 0.5:
      polarity = 'Compound'
    
  print(i, ss[i])
  
#polarity
  

neg 0.118
neu 0.642
pos 0.241
compound 0.3182


'neutral'

In [6]:
se = pd.Series(listy)
se1 = pd.Series(sentiment)
data['polarity'] = se.values
data['Sentiment'] = se1.values

display(data.head(100))

,Tweets,polarity,Sentiment
0,RT @ZeeNewsHindi: VIDEO: जब विंग कमांडर अभिनंद...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
1,RT @Republic_Bharat: UNSC से आज भारत को मिल सक...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
2,"RT @PMOIndia: Dr. Saud Mohammed Al-Sati, Saudi...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
3,RT @Shakra_Kulish: Mohan Das Karam Chand Gandh...,"{'neg': 0.118, 'neu': 0.882, 'pos': 0.0, 'comp...",neutral
4,@QararaRasha Life is complex and so is interna...,"{'neg': 0.204, 'neu': 0.796, 'pos': 0.0, 'comp...",neutral
5,@bilalahmed481 That is Pakistani occupied kahm...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
6,Welcome to India wc abhinandan sir,"{'neg': 0.0, 'neu': 0.625, 'pos': 0.375, 'comp...",neutral
7,@swiftsIytherin yes listen remember when Pakis...,"{'neg': 0.243, 'neu': 0.648, 'pos': 0.109, 'co...",neutral
8,RT @CarefreeMunda: Jab se hamare mulk walo ne ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",neutral
9,RT @Babu_Bhaiyaa: India entered Pak for reveng...,"{'neg': 0.323, 'neu': 0.677, 'pos': 0.0, 'comp...",neutral


In [7]:
data['Sentiment'].describe()

count         100
unique          1
top       neutral
freq          100
Name: Sentiment, dtype: object